## CLIP scene classification - finding more context

Loading scene Labels

In [ ]:
import pandas as pd

In [ ]:
df= pd.read_excel("/content/CLIP_labels_IC.xlsx")

In [ ]:
df.head()

In [ ]:
upd_df=df['scene']

In [ ]:
upd_df.head()

In [ ]:
scene_labels=upd_df.to_list()

In [ ]:
scene_labels

In [ ]:
len(scene_labels)

23

In [ ]:
#scene_labels=[f'the image is depicting scene of {label}' for label in scene_labels_raw]

In [ ]:
#scene_labels

initialize CLIP

In [ ]:
# initialization
from transformers import CLIPProcessor, CLIPModel

model_id = "openai/clip-vit-large-patch14"

processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id)

## Test for Multiple Images

In [ ]:
df_surv = pd.read_excel('/content/blip-clip-REPEATED_surveillance_dataset_captioned.xlsx')

In [ ]:
df_surv.head()

In [ ]:
df_surv_upd = df_surv['URL']

In [ ]:
df_surv_upd.head()

In [ ]:
url_lst_images=df_surv_upd.to_list()

In [ ]:
len(url_lst_images)

69

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
final_dict={}

In [ ]:
from PIL import Image
import requests

In [ ]:
# url = "https://galelawgroup.com/wp-content/uploads/2022/02/robberyfaq.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
for url in tqdm(url_lst_images):
  try:
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(text=scene_labels, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

    context= scene_labels[probs.argmax(-1)]
    final_dict[url] = context
  except:
    continue
  else:
    print(context)

## Save results to Excel sheet

In [ ]:
df_final = pd.DataFrame([final_dict]).transpose()

In [ ]:
df_final.head()

,0
https://houstoncriminalfirm.com/wp-content/uploads/2019/06/Canva-_-Theft-900x600.jpg,Shoplifting
https://www.careylawoffice.com/wp-content/uploads/2021/12/Robbery-Theft-and-Assault-Crimes-Against-People-vs.-Property.jpeg,Suspicious activity
https://www.altshulerlaw.com/wp-content/uploads/Robbery-and-Theft-Abound-In-Tennessee-300x200.jpg,Robbery
https://www.cmcdefense.com/images/blog/California-Burglary-Law-1-min.jpg,Robbery
https://rhdefense.com/wp-content/uploads/2017/05/robbery-theft.jpg,Robbery


In [ ]:
df_final.count()

0    68
dtype: int64

In [ ]:
file_name = 'context_blip-clip.xlsx'

In [ ]:
df_final.to_excel(file_name)

## Test for one Images

In [ ]:
inputs = processor(text=scene_labels, images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


In [ ]:
context_raw= scene_labels[probs.argmax(-1)]

In [ ]:
context= 'the image is depicting scene of '+ context_raw

Get the context

In [ ]:
context

'the image is depicting scene of Shooting'

## BLIP- Generate Initial caption

In [ ]:
from transformers import pipeline
image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

In [ ]:
url='https://galelawgroup.com/wp-content/uploads/2022/02/robberyfaq.jpg'

In [ ]:
caption = image_to_text(url, max_new_tokens=200)
initial_caption= caption[0]['generated_text']

In [ ]:
initial_caption

'there is a woman holding a gun in front of a man'

## Combine initial caption with context to generate final caption using LLM

In [ ]:
!pip install -q --upgrade langchain

##using langchain to produce final caption for multiple images

In [ ]:
!pip install -q langchain-google-genai


In [ ]:
!pip install -q langchain

In [ ]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
from google.generativeai.types.safety_types import HarmBlockThreshold, HarmCategory

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest", google_api_key=GOOGLE_API_KEY, temperature=0.2, top_p=1, top_k=1,
                             safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,

    }, #convert_system_message_to_human=True
      )

In [ ]:
template="""You are an advanced image captioning AI assistant for surveillance related images. Your task is to refine and improve an initial image caption using relevant contextual information provided. You will receive two inputs:
Input 1: {initial_caption} - This is the initial caption for the image, most likely grammatically incorrect and incomplete sentence, generated by a separate not so good image captioning model.
Input 2: {context} - This is the contextual information that provides more details about the background
Your goal is to take the initial caption and the additional context, and produce a new, refined caption that incorporates the contextual details. Please do not speculate things which are not provided. The final caption should be grammatically correct.
Please output only the final caption.
Here are some examples:
User: initial_caption- arafed car on its side after a crash in a residential area
	  context- the image is depicting scene of road accident
AI:   final_caption- A car has crashed and flipped onto its side in a residential area.

User: initial_caption-a man in a black hoodie holding a knife in a hallway
	  context-the image is depicting scene of attack
AI:   final_caption- A man in a black hoodie is attacking someone with a knife in a hallway.

User: initial_caption-there is a man holding a bag and a woman looking at a cell phone
	  context-the image is depicting scene of stealing
AI:   final_caption- A man is stealing a bag from a woman who is looking at her cell phone.
"""

In [ ]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=["initial_caption", "context"],
)

## load master dataframe

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/blip-clip-REPEATED_surveillance_dataset_captioned.xlsx')

In [ ]:
df.head(2)

,URL,Image,captions_ground_truth,1 capt_cnn-lstm,2 capt_cnn-lstm-att,3 capt_cnn-transf,4 caption_blip,context_raw,context,5 caption_llm
0,https://houstoncriminalfirm.com/wp-content/upl...,NaN,a woman holding a bag and a man secretly tryin...,the woman in the blue shirt is looking down a...,there are two women playing the nintendo wii g...,a woman in a red jacket is sitting on a red bench,there is a woman holding a bag and a man looki...,Stealing,the image is depicting scene of Stealing,NaN
1,https://www.careylawoffice.com/wp-content/uplo...,NaN,man in a black hoodie and black trying to brea...,the man is wearing black coat and hat is look...,a woman wearing a black jacket and yellow hat ...,a man in a black hat and a black coat is playi...,arafed man in a black hood and black jacket ho...,Suspicious activity,the image is depicting scene of Suspicious act...,NaN


In [ ]:
df_upd=df['URL']

In [ ]:
url_list=df_upd.to_list()

In [ ]:
len(url_list)

69

In [ ]:
df_upd.head(3)

,URL,captions_ground_truth
0,https://houstoncriminalfirm.com/wp-content/upl...,
1,https://www.careylawoffice.com/wp-content/uplo...,
2,https://www.altshulerlaw.com/wp-content/upload...,


In [ ]:
df_upd.info()

<class 'pandas.core.series.Series'>
RangeIndex: 69 entries, 0 to 68
Series name: URL
Non-Null Count  Dtype 
--------------  ----- 
69 non-null     object
dtypes: object(1)
memory usage: 680.0+ bytes


In [ ]:
df_upd=df_upd.to_frame()

In [ ]:
df_upd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 0 to captions_ground_truth
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   URL                    70 non-null     object
 1   captions_ground_truth  70 non-null     object
dtypes: object(2)
memory usage: 3.7+ KB


In [ ]:
df_upd['captions_ground_truth']=''

In [ ]:
df_upd.head(3)

,URL,captions_ground_truth
0,https://houstoncriminalfirm.com/wp-content/upl...,
1,https://www.careylawoffice.com/wp-content/uplo...,
2,https://www.altshulerlaw.com/wp-content/upload...,


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import time

In [ ]:
for index, row in tqdm(df_upd.iterrows()):
  try:
    initial_caption= row['4 caption_blip']
    context= row['context']

    prompt=prompt_template.format(initial_caption=initial_caption, context=context)
    response = llm.invoke(prompt)

    time.sleep(5)
    df.at[index, 'final_caption'] = response.content
  except:
    continue
  else:
    print(response.content)

## save to excel

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
file_name = 'captioned_llm-crime.xlsx'

In [ ]:
df.to_excel(file_name)

##using gemini vision to genrate ground truth

In [ ]:
import requests
from IPython.display import Image
# image_url = "https://galelawgroup.com/wp-content/uploads/2022/02/robberyfaq.jpg"
# content = requests.get(image_url).content
# Image(content)

In [ ]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.0-pro-vision-latest",
    safety_settings={
         HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
from tqdm.notebook import tqdm

url_list is defined in above section

In [ ]:
prompt_text="Please generate one sentence caption for this image, which should describe this image without leaving important details"

In [ ]:
final_dict={}

In [ ]:
import time

In [ ]:
for url in tqdm(url_list):
  try:
    message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": prompt_text,
        },
        {"type": "image_url", "image_url": url},
    ])

    response= llm.invoke([message])
    final_dict[url] = response.content
  except:
    continue
  else:
    print(response.content)

## save as excel

In [ ]:
df_final = pd.DataFrame([final_dict]).transpose()

In [ ]:
df_final.head()

,0
https://houstoncriminalfirm.com/wp-content/uploads/2019/06/Canva-_-Theft-900x600.jpg,A thief is pickpocketing a woman's purse whil...
https://www.careylawoffice.com/wp-content/uploads/2021/12/Robbery-Theft-and-Assault-Crimes-Against-People-vs.-Property.jpeg,A burglar wearing a black hoodie and mask is ...
https://www.altshulerlaw.com/wp-content/uploads/Robbery-and-Theft-Abound-In-Tennessee-300x200.jpg,A thief wearing a black hoodie is robbing a w...
https://www.cmcdefense.com/images/blog/California-Burglary-Law-1-min.jpg,A burglar wearing a black ski mask is attempt...
https://rhdefense.com/wp-content/uploads/2017/05/robbery-theft.jpg,A masked man is holding a gun and pointing it...


In [ ]:
df_final.count()

0    69
dtype: int64

In [ ]:
file_name = 'ground_truth_captions_crime.xlsx'

In [ ]:
df_final.to_excel(file_name)

## for one example

In [ ]:
initial_caption="there is a man that is opening the door of a car"
context="the image is depicting scene of Suspicious activity"


In [ ]:
prompt=prompt_template.format(initial_caption=initial_caption, context=context)

In [ ]:
response = llm.invoke(prompt)

In [ ]:
response.content

'A man is acting suspiciously and opening the door of a car.'

## using gemi api

In [ ]:
!pip -q install google-generativeai

In [ ]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}

In [ ]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [ ]:
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": ["You are an advanced image captioning AI assistant. Your task is to refine and improve an initial image caption using relevant contextual information provided. You will receive two inputs:\nInput 1: {initial_caption} - This is the original caption for the image, likely generated by a different system without additional context.\nInput 2: {context} - This is the contextual information that provides more details about the background\nYour goal is to take the initial caption and the additional context, and produce a new, refined caption that incorporates the contextual details in a natural and coherent way. Please do not add any extra info/ details."]
  },
  {
    "role": "model",
    "parts": ["{initial_caption}: A group of people are standing in a kitchen.\n{context}: The people are cooking a meal together. They are laughing and having a good time.\n{refined_caption}: A group of people are cooking a meal together in a kitchen. They are laughing and having a good time."]
  },
  {
    "role": "user",
    "parts": ["initial_caption : A masked man and a woman bag\ncontext: The image is depicting scene of roobbery"]
  },
  {
    "role": "model",
    "parts": ["{initial_caption}: there is a man holding a gun in front of a woman\n{context}: the image is depicting scene of shooting\n{refined_caption}: A man is shooting a woman with a gun."]
  },
])


In [ ]:
USER_INPUT="initial_caption : there is a man bag and a woman. context: the image is depicting scene of robbery"

In [ ]:
convo.send_message(USER_INPUT)
print(convo.last.text)

{initial_caption}: A group of people are standing in a kitchen.
{context}: The people are cooking a meal together. They are laughing and having a good time.
{refined_caption}: A group of people are cooking a meal together in a kitchen. They are laughing and having a good time.


In [ ]:
convo.history

[parts {
   text: "You are an advanced image captioning AI assistant. Your task is to refine and improve an initial image caption using relevant contextual information provided. You will receive two inputs:\nInput 1: {initial_caption} - This is the original caption for the image, likely generated by a different system without additional context.\nInput 2: {context} - This is the contextual information that provides more details about the background\nYour goal is to take the initial caption and the additional context, and produce a new, refined caption that incorporates the contextual details in a natural and coherent way. Please do not add any extra info/ details."
 }
 role: "user",
 parts {
   text: "{initial_caption}: A group of people are standing in a kitchen.\n{context}: The people are cooking a meal together. They are laughing and having a good time.\n{refined_caption}: A group of people are cooking a meal together in a kitchen. They are laughing and having a good time."
 }
 role